#### Preface

In this short tutorial, the tuning process for further evaluation is demonstrated. For this purpose, datasets from the [chunking_evaluation](https://github.com/brandonstarxel/chunking_evaluation) framework will be used.

#### Packages installation 

In [19]:
!pip install "git+https://github.com/panalexeu/horchunk.git"
!pip install requests
!pip install chroma
!pip install numpy

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Cloning https://github.com/panalexeu/horchunk.git to /tmp/pip-req-build-l3fpo03i
  Running command git clone --filter=blob:none --quiet https://github.com/panalexeu/horchunk.git /tmp/pip-req-build-l3fpo03i
  Resolved https://github.com/panalexeu/horchunk.git to commit 4fd7e6936057689b3677d92f7a1be398cbf388fd
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


#### Loading datasets from git

In [20]:
import requests 

links_dict = dict(
    wikitexts='https://raw.githubusercontent.com/brandonstarxel/chunking_evaluation/refs/heads/main/chunking_evaluation/evaluation_framework/general_evaluation_data/corpora/wikitexts.md',
    chatlogs='https://raw.githubusercontent.com/brandonstarxel/chunking_evaluation/refs/heads/main/chunking_evaluation/evaluation_framework/general_evaluation_data/corpora/chatlogs.md',
    finance='https://raw.githubusercontent.com/brandonstarxel/chunking_evaluation/refs/heads/main/chunking_evaluation/evaluation_framework/general_evaluation_data/corpora/finance.md',
    pubmed='https://raw.githubusercontent.com/brandonstarxel/chunking_evaluation/refs/heads/main/chunking_evaluation/evaluation_framework/general_evaluation_data/corpora/pubmed.md',
    state_of_the_union='https://raw.githubusercontent.com/brandonstarxel/chunking_evaluation/refs/heads/main/chunking_evaluation/evaluation_framework/general_evaluation_data/corpora/state_of_the_union.md'
) 


def load_datasets(links_dict: dict) -> dict:
    data = dict()
    for name, link in links_dict.items():
        response = requests.get(link)
        data[name] = response.text 
        
    return data 

datasets = load_datasets(links_dict)
for key in datasets.keys():
    print(f'{key}: {len(datasets[key].split())} words')

wikitexts: 22406 words
chatlogs: 5968 words
finance: 116860 words
pubmed: 75846 words
state_of_the_union: 8468 words


#### Tuning

Let's start by instantiating WindowTuner.

In [21]:
from horchunk.chunkers import WindowTuner
from chromadb.utils import embedding_functions

ef = embedding_functions.DefaultEmbeddingFunction() # all-MiniLM-L6-v2
tuner = WindowTuner(ef) 

In this tutorial, we will tune the minimum threshold for document of 3 sentences in size. We will repeat this process for every dataset. The identified thresholds will then be averaged to determine a single, generalized threshold value.

In [22]:
thresholds = []
DEPTH = 4

**wikitexts**

In [23]:
from horchunk.splitters import SentenceSplitter 

splitter = SentenceSplitter(text=datasets['wikitexts'])
splits = splitter()
res = tuner(splits, depth=DEPTH)
thresholds.append(res)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1090/1090 [01:50<00:00,  9.90it/s]


1089 chunks formed

Values range: [0.013122331351041794 ... 1.0]

Steps to tune: 10

--------------------------------

...

chunk: The route continues to travel through farmland for sixteen miles ( 26 km ) , where it enters Morrill County 
.The road crosses over Pumpkin Creek four times , and enters the unincorporated community of <unk> .

Type 'k' to raise thresh, or 'j' - to lower it, then press 'Enter':  j


dist: 0.8072338700294495

chunk: Barker sometimes incorporated portraits of her fellow parishioners in her religious works .She was described
by Canon Ingram Hill as " one of the pillars " of St.

Tuning ended, thresh value: 0.8072338700294495 = 0.80

**chatlogs**

In [24]:
splitter = SentenceSplitter(text=datasets['chatlogs'])
splits = splitter()
res = tuner(splits, depth=DEPTH)
thresholds.append(res)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 257/257 [00:28<00:00,  8.91it/s]


257 chunks formed

Values range: [0.05278143659234047 ... 1.000000238418579]

Steps to tune: 8

--------------------------------

...

chunk: The Cross Bones Graveyard - a medieval graveyard that became known as an outcast cemetery with a history 
connected to the sex industry, now a memorial garden.\n\n8.Highgate Cemetery - a beautiful Victorian graveyard 
known for its Gothic architecture and where many famous people, including Karl Marx, are buried.\n\n9.

Type 'k' to raise thresh, or 'j' - to lower it, then press 'Enter':  k


dist: 0.8727983236312866

chunk: Hornsey Town Hall Arts Centre - a grade II listed building with a rich history and home to many art 
exhibitions and events.\n\n3.Clapham Grand - a historic Victorian theatre that has been transformed into a music 
venue, cinema and a comedy club.\n\n4.

Tuning ended, thresh value: 0.8727983236312866 = 0.87

**finance**

In [25]:
splitter = SentenceSplitter(text=datasets['finance'])
splits = splitter()
res = tuner(splits, depth=DEPTH)
thresholds.append(res)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6320/6320 [10:24<00:00, 10.12it/s]


3593 chunks formed

Values range: [0.024115130305290222 ... 1.0]

Steps to tune: 11

--------------------------------

...

chunk: net charges and adjustments | 4.3 | 0.8 | 0.7 | 5.8payments and other1 | -15.0 (15.0) | -4.1 (4.1) | -3.5 
(3.5) | -22.6 (22.6)

Type 'k' to raise thresh, or 'j' - to lower it, then press 'Enter':  j


dist: 0.8559920191764832

chunk: as of december 31, 2015, the company did not have any direct investments in commercial real estate or direct
commercial mortgages or any material holdings of derivative investments (other than equity index put option 
contracts as discussed in item 8, 201cfinancial statements and supplementary data 201d - note 4 of notes to 
consolidated financial statements) or securities of issuers that are experiencing cash flow difficulty to an extent
that the company 2019s management believes could threaten the issuer 2019s ability to meet debt service payments, 
except where other-than-temporary impairments have been recognized.the company 2019s investment portfolio includes 
structured commercial mortgage-backed securities (201ccmbs 201d) with a book value of $264.9 million and a market 
value of $266.3 million.

Tuning ended, thresh value: 0.8559920191764832 = 0.85

**pubmed**

In [26]:
splitter = SentenceSplitter(text=datasets['pubmed'])
splits = splitter()
res = tuner(splits, depth=DEPTH)
thresholds.append(res)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4458/4458 [06:52<00:00, 10.81it/s]


4393 chunks formed

Values range: [-0.13710103929042816 ... 1.000000238418579]

Steps to tune: 12

--------------------------------

...

chunk: Cases of Lateral TransferOf the 205 alignments, two were found to strongly reject the most accepted tree 
(topology 5) as well as all other topologies tested.

Type 'k' to raise thresh, or 'j' - to lower it, then press 'Enter':  j


dist: 0.4131526052951813

chunk: 1997).Recombination assays.

Tuning ended, thresh value: 0.4131526052951813 = 0.41

**state_of_the_union**

In [ ]:
splitter = SentenceSplitter(text=datasets['state_of_the_union'])
splits = splitter()
res = tuner(splits, depth=DEPTH)
thresholds.append(res)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 658/658 [01:00<00:00, 10.83it/s]


656 chunks formed

Values range: [0.10046437382698059 ... 0.9999999403953552]

Steps to tune: 9

--------------------------------

dist: 0.8220882415771484

chunk: Some came when famine struck, like my ancestral family in Ireland.Some to flee persecution, to chase dreams 
that are impossible anywhere but here in America.

#### Averaging thresh values 

The calculated average threshold value will be used for evaluation over the general dataset in the `evaluation.ipynb`notebook.

In [18]:
import numpy as np

print(len(thresholds), thresholds)
thresh = np.mean(thresholds)
print(f'avg. thresh: {thresh} = {round(thresh, 2)}')

5 [0.7941413521766663, 0.6194384098052979, 0.5499992966651917, 0.5951321125030518, 0.4767434597015381]
avg. thresh: 0.6070909261703491 = 0.61
